First we shall join the newly split files with their original counterparts:

In [7]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
f = "/home/nikolal/projects/parlaspeech/transcripts_normalized.txt"
old = pd.read_table(f, names=["unknown1", "path", "sentence"],
        #nrows=2000
        )

old["path"] = "/home/peterr/macocu/task6_speech/data/" + \
    old.path.str.replace("flac/", "") + ".wav"

old = old[["path", "sentence"]]
old.head()

,path,sentence
0,/home/peterr/macocu/task6_speech/data/00000001.flac.wav,poštovane kolegice i kolege zastupnici molio bi da nastavimo sa radom sjednice
1,/home/peterr/macocu/task6_speech/data/00000002.flac.wav,prvi klub koji se javio za stanku je klub esdepea
2,/home/peterr/macocu/task6_speech/data/00000003.flac.wav,u njihovo ime predstavnik bojan glavašević
3,/home/peterr/macocu/task6_speech/data/00000004.flac.wav,hvala gospodine predsjedniče
4,/home/peterr/macocu/task6_speech/data/00000005.flac.wav,poštovani gospodine predsjedniče hrvatskog sabora uvažene kolegice i kolege klub zastupnika esdepea tražio je stanku zbog problema na koji smo upozorili na konstituirajućoj sjednici sabora


In [8]:
new = pd.read_csv("14_transcripts_ready.csv")
new.head(1)

,path,sentence_uncut,original_file,words,words_start_times,words_end_times,duration,sentence
0,/home/peterr/macocu/task6_speech/data/00015807a.flac.wav,dakle uvaženi kolega ja sam pozorno bio pratio vaše izlaganje i spomenuli ste riječ outsourcing prilikom vanjske produkcije,/home/peterr/macocu/task6_speech/data/00015807.flac.wav,"['dakle', 'uvaženi', 'kolega', 'ja', 'sam', 'pozorno', 'bio', 'platio']","[0.561111111111111, 1.603174603174603, 2.2043650793650795, 2.785515873015873, 3.005952380952381, 3.5269841269841264, 4.10813492063492, 4.328571428571428]","[0.8416666666666666, 2.044047619047619, 2.4648809523809523, 2.8456349206349203, 3.166269841269841, 3.987896825396825, 4.228373015873016, 4.669246031746032]",5.05,dakle uvaženi kolega ja sam pozorno bio pratio


# Reasoning:

From the `new` dataframe we will get the filenames of the original files. We will delete these from the `old` dataframe, as otherwise we would have the same data numerous times.

In [3]:
new_paths = new.original_file.unique()
new_paths

array(['/home/peterr/macocu/task6_speech/data/00015807.flac.wav',
       '/home/peterr/macocu/task6_speech/data/00000154.flac.wav',
       '/home/peterr/macocu/task6_speech/data/00002597.flac.wav', ...,
       '/home/peterr/macocu/task6_speech/data/00006520.flac.wav',
       '/home/peterr/macocu/task6_speech/data/00018811.flac.wav',
       '/home/peterr/macocu/task6_speech/data/00009875.flac.wav'],
      dtype=object)

## Removing unneded files

In [10]:
import os
to_remove = old.path[old.path.isin(new_paths)].values
for file in to_remove:
    os.remove(file)

In [4]:
c = old.path.isin(new_paths)
old = old.loc[~c, :].reset_index(drop=True)
old.head(3)

,path,sentence
0,/home/peterr/macocu/task6_speech/data/00000001.flac.wav,poštovane kolegice i kolege zastupnici molio bi da nastavimo sa radom sjednice
1,/home/peterr/macocu/task6_speech/data/00000002.flac.wav,prvi klub koji se javio za stanku je klub esdepea
2,/home/peterr/macocu/task6_speech/data/00000003.flac.wav,u njihovo ime predstavnik bojan glavašević


In [45]:
concat_df = pd.concat([old, new], ignore_index=True).sort_values("path").reset_index(drop=True)
concat_df.iloc[:, slice(0, 2)].to_csv("15_new_DS.csv", index=False)

# Proceeding with the training:

In [47]:
df = concat_df

LIM = int(df.shape[0] * 0.8)
common_voice_train_df = df.loc[0:LIM, :].copy()
common_voice_test_df = df.loc[LIM:, :].copy()


common_voice_train_df.reset_index(drop=True, inplace=True)
common_voice_test_df.reset_index(drop=True, inplace=True)


import datasets
from datasets import load_dataset, load_metric, Audio
def load_audio(path):
    return datasets.Audio(sampling_rate=16000).decode_example(path)

# Adding audio
common_voice_train_df.loc[:, "audio"] = common_voice_train_df.path.apply(load_audio)
common_voice_test_df.loc[:, "audio"] = common_voice_test_df.path.apply(load_audio)

# Initiating a dataset from pandas dataframe
common_voice_train_dataset = datasets.Dataset.from_pandas(common_voice_train_df)
common_voice_test_dataset = datasets.Dataset.from_pandas(common_voice_test_df)


from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
    "./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)

import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)








from transformers import Trainer
from transformers import TrainingArguments
from transformers import Wav2Vec2ForCTC
from typing import Any, Dict, List, Optional, Union
from dataclasses import dataclass, field
import torch


def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(
        audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch


common_voice_train_mapped = common_voice_train_dataset.map(
    prepare_dataset, remove_columns=common_voice_train_dataset.column_names)
common_voice_test_mapped = common_voice_test_dataset.map(
    prepare_dataset, remove_columns=common_voice_test_dataset.column_names)

file ./config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Parameter 'function'=<function prepare_dataset at 0x7fde5062bb80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/29853 [00:00<?, ?ex/s]

  0%|          | 0/7464 [00:00<?, ?ex/s]

In [50]:
repo_name = "15_"
wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

max_input_length_in_sec = 20
common_voice_train_mapped = common_voice_train_mapped.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

model.freeze_feature_extractor()

from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=4,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=1,
  push_to_hub=False,
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train_mapped,
    eval_dataset=common_voice_test_mapped,
    tokenizer=processor.feature_extractor,
)

trainer.train()

  0%|          | 0/30 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/facebook/wav2vec2-xls-r-300m/resolve/main/config.json from cache at /home/peterr/.cache/huggingface/transformers/dabc27df63e37bd2a7a221c7774e35f36a280fbdf917cf54cadfc7df8c786f6f.a3e4c3c967d9985881e0ae550a5f6f668f897db5ab2e0802f9b97973b15970e6
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activ

Step,Training Loss,Validation Loss


/home/peterr/anaconda3/lib/python3.8/site-packages/transformers/trainer.py:1355: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
